# Tutorial: STL generation and output

In this tutorial, we will generate a water-tight STL file, visualize the STL, and output to file

The pumapy STL generation uses the Lewiner marching cubes implementation from scikit-image. The C++ version of PuMA also includes an implementation of the original and Lewiner marching cubes. 

The Lewiner marching cubes method is used to generate STLs because the surface is guaranteed to be topologically correct (i.e watertight). The original marching cubes is suitable for visualization purposes, but had ambiguitites that resulted in small holes in the surface mesh. 

In [1]:
# ONLY FOR GOOGLE COLAB: Run this cell only the first time you open a tutorial
if 'google.colab' in str(get_ipython()):
    !pip install -q condacolab
    import condacolab
    condacolab.install()
    !conda install -c fsemerar puma

First, we must import puma:

In [2]:
import numpy as np
import os
import sys
import pumapy as puma
import pyvista as pv

Next we must either import or generate a material. Here we will import a material. If you prefer to generate a material instead, refer to the material generation tutorials. 

In [3]:
# Next we will import an example tomography file of size 200^3 and voxel length 1.3e-6
ws_imported = puma.import_3Dtiff(puma.path_to_example_file("200_fiberform.tif"), 1.3e-6)

Importing /Users/fsemerar/Documents/PuMA_playground/puma-dev/python/pumapy/data/200_fiberform.tif ... Done


The workspace above has not been segmented yet. In the case of generated workspaces, the generated material is typically stored for a grayscale [128,255] and the void phase is stored between [0,127]. 

In this case, the appropriate grayscale cutoff for the imported tomography sample is 90, such that [90,255] is material and [0,89] is the void. These values will be different for each tomography image. 

It is usually better to generate an STL based on a non-segmented material. This is because the segmentation process removes most of the information defining the surface from the tomography data. As an illustration, the code below will visualze the segmented and non-segmented versions of the imported tomography file. 

In [4]:
ws_segmented = ws_imported.copy()
ws_segmented.binarize(90)

In [9]:
p = pv.Plotter(shape=(1, 2))
p.subplot(0, 0)
p.add_text("Raw grayscales")
puma.render_volume(ws_imported, notebook=True, add_to_plot=p, plot_directly=False)
p.subplot(0, 1)
p.add_text("Segmented")
puma.render_volume(ws_segmented, notebook=True, add_to_plot=p, plot_directly=False)
p.update_scalar_bar_range(1)
p.show()

ViewInteractiveWidget(height=1200, layout=Layout(height='auto', width='100%'), width=1920)

In [12]:
p = pv.Plotter(shape=(1, 2))
p.subplot(0, 0)
p.add_text("Raw grayscales")
puma.render_contour(ws_imported, cutoff=(90, 255), notebook=True, add_to_plot=p, plot_directly=False)
p.subplot(0, 1)
p.add_text("Segmented")
puma.render_contour(ws_segmented, cutoff=(1, 1), notebook=True, add_to_plot=p, plot_directly=False)
p.show()

ViewInteractiveWidget(height=1200, layout=Layout(height='auto', width='100%'), width=1920)

As you can see, the non-segmented image provides a far superior surface representation. Because of this, we will use the non-segmented material for the STL generation. 

In order to output an STL, one must simply call the puma.export_stl function, as shown below:

Let's now export the STL. We need to first specify a path:

In [13]:
export_path = "../tests/out"  # CHANGE THIS PATH

In [14]:
filename = "FiberForm-nonsegmented.stl"
cutoff = (90,255)  # tuple containing the grayscale range of the material to be exported
flag_closed_edges = True  # flag for whether to cap the regions of the material that intersect with the domain boundary

puma.export_stl(os.path.join(export_path, filename), ws_imported, cutoff, flag_closed_edges)

Exporting ../tests/out/FiberForm-nonsegmented.stl ... Done


True

If you have no choice but to use a segmented image to export an stl, the flag_gauissian option can be set to True, which will apply a gauissian blur to the image prior to running the marching cubes algorithm. This will make the surface appear more smooth, but will also blur out any sharp features that may be present in the actual microstructure. 

In [15]:
filename = "FiberForm-segmented.stl"
cutoff = (1,1)  # tuple containing the grayscale range of the material to be exported
flag_closed_edges = True  # flag for whether to cap the regions of the material that intersect with the domain boundary

puma.export_stl(os.path.join(export_path, filename), ws_segmented, cutoff, flag_closed_edges, flag_gaussian=False)

filename = "FiberForm-segmented-gaussian.stl"
puma.export_stl(os.path.join(export_path, filename), ws_segmented, cutoff, flag_closed_edges, flag_gaussian=True)

Exporting ../tests/out/FiberForm-segmented.stl ... Done
Exporting ../tests/out/FiberForm-segmented-gaussian.stl ... Done


True

The STLs can be imported into other software for simulation or visualization purposes